## Create ZARR store

In this notebook i will load raw data in xarray and then save on a zarr store for future work. (Its faster)

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cmaps
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cf
import datetime
import geopandas as gpd
import xesmf as xe
import glob
import regionmask
import xgcm
import dask
import os
from dask.diagnostics import ProgressBar

In [2]:
def fix_crocotime(DATA, YORIG, time_name='time'):
    """
    Grab simulation time and transform to datetime objects based on given YORIG

    Args:
        DATA (XDataset, XDataArray): CROCO simulation data, with "time" coordinate 
        YORIG (str): Given reference date

    Returns:
        XDataset, XDataArray: Data with fixed time coordinate
    """
    ORIG = pd.to_datetime(YORIG)
    if time_name=='time':
        new_time = pd.to_datetime([datetime.timedelta(seconds=t.item())+ORIG
                                for t in DATA[time_name]])
    else:
        new_time = DATA[time_name]+ORIG
        
    DATA[time_name] = new_time
    return DATA.sortby(time_name)

def center_grid(data,variables):
    """
    This function grabs a croco model outputs and moves the 
    arakawa-C edges variables (like u and v) to the center of the grid.

    Args:
        data (xarray): input dataset
        variables (list): list of variables to transform

    Returns:
        xarray: dataset with variables in the center of the grid 
    """
    for v in variables:
        if 'eta_v' in data[v].dims:
            x = data[v].interp(eta_v=data.eta_rho.values)
            x = x.rename({'eta_v':'eta_rho'})
            data = data.drop(v)
            data[v]=x
        if 'eta_u' in data[v].dims:
            x = data[v].interp(eta_u=data.eta_rho.values)
            x = x.rename({'eta_u':'eta_rho'})
            data = data.drop(v)
            data[v]=x
        if 'xi_u' in data[v].dims:
            x = data[v].interp(xi_u=data.xi_rho.values)
            x = x.rename({'xi_u':'xi_rho'})
            data = data.drop(v)
            data[v]=x
        if 'xi_v' in data[v].dims:
            x = data[v].interp(xi_v=data.xi_rho.values)
            x = x.rename({'xi_v':'xi_rho'})
            data = data.drop(v)
            data[v]=x
    return data

def load_croco(paths, YORIG, time_name='time', variables=None, **kwargs):
    """
    Loading function for reading raw CROCO/ROMS outputs into
    xarray objects. **kwargs are passed to xarray.open_dataset() function

    Args:
        paths (str, list): _description_
        YORIG (str): _description_
        variables (list, optional): _description_. Defaults to None.

    Raises:
        ValueError: If input path is not a string or list of strings

    Returns:
        xarray: loaded croco data
    """
    if type(paths) == str:
        if "*" in paths:
            data = xr.open_mfdataset(paths,
                                    concat_dim='time',
                                    combine='nested',
                                    **kwargs)
            data = fix_crocotime(data, YORIG)
        else:
            data = xr.open_dataset(paths, **kwargs)
            data = fix_crocotime(data, YORIG, time_name=time_name)
    elif type(paths) == list:
        data=[]
        for p in paths:
            d = xr.open_dataset(p, **kwargs)
            d = fix_crocotime(d,YORIG, time_name=time_name)
            data.append(d)
        data = xr.concat(data,'time')
    else:
        raise ValueError('Path to croco output should only be a string or a list of strings.')
    if variables==None:
        data = center_grid(data, data.keys())
    else:
        data = center_grid(data, variables)[variables]
    return data.sortby('time')

def croco_sellonlatbox(data, lonmin, lonmax, latmin, latmax):
    """
    This functions grabs a croco output and slice it to the
    desired latlon bounds. 
    Only works for rho point centered variables (like temp, salt, zeta, etc)
    The arakawa-C edges variables like zonal and meridional currents must first
    be horizontally interpolated to the grid center.

    Args:
        data (xarray): loaded dataset (centered in rho points) as xarray object
        lonmin (float): min longitude
        lonmax (float): max longitude
        latmin (float): min latitude
        latmax (float): max latitude

    Returns:
        data: sliced data to the user defined latlon box 
    """
    data = data.sortby('eta_rho').sortby('xi_rho')
    geoindex = ((data.lon_rho > lonmin) & (data.lon_rho<lonmax) & (data.lat_rho>latmin) & (data.lat_rho < latmax)).load()
    geoindex = np.argwhere(geoindex.values)
    xmin = min(geoindex[:,1])
    xmax = max(geoindex[:,1])
    ymin = min(geoindex[:,0])
    ymax = max(geoindex[:,0])
    data = data.sel(eta_rho=slice(ymin,ymax), xi_rho=slice(xmin,xmax))
    return data




In [3]:
grid = xr.open_dataset('data/CROCO/OUTPUT/TESTSIM/3HERA5_GLORYS12V1/testsim_grd.nc').squeeze().load()
grid = grid[['h','xi_rho','eta_rho','lon_rho','lat_rho','x_rho','y_rho','mask_rho']]
grid['lon_rho'] = (grid.lon_rho+180)%360-180


In [5]:
paths = sorted(glob.glob('data/CROCO/CROCO_FILES/testsim/24H/nomask/*blk*'))
aforcing = [xr.open_dataset(p, chunks=dict(bulk_time=1))
            for p in paths]
aforcing = xr.concat(aforcing, 'bulk_time')
aforcing = fix_crocotime(center_grid(aforcing, aforcing.variables), '1900-01-01 00:00:00', time_name='bulk_time')
aforcing = aforcing.drop_duplicates('bulk_time').rename({'bulk_time':'time'})


In [6]:
aforcing.to_zarr('data/ZARR/testim_aforc_nomask', consolidated=True, mode='w')
grid.to_zarr('data/ZARR/testim_aforc_nomask/', consolidated=True, mode='a')

/ceaza/lucas/miniconda3/envs/main/lib/python3.8/site-packages/xarray/core/dataset.py:2036: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(


In [4]:


x = xr.open_dataset('data/CROCO/OUTPUT/TESTSIM/3HERA5_GLORYS12V1/testsim_avg_Y2006M1.nc')[['hc','Cs_r']]
hc,Cs_r = x.hc.item(),x.Cs_r
del x

def rhopoints_depths(h, zeta, s_rho, Cs_r, hc, vtransform=2):
    """ Compute depth of roms sigma levels.

    Args:
        h (_type_): _description_
        zeta (_type_): _description_
        s_rho (_type_): _description_
        Cs_r (_type_): _description_
        hc (_type_): _description_
        vtransform (int, optional): _description_. Defaults to 2.

    Returns:
        _type_: _description_
    """
    if vtransform==1:
        Z_rho = hc*(s_rho-Cs_r)+Cs_r*h
        z_rho = Z_rho+zeta*(1+Z_rho/h)
        return z_rho
    else:
        Z_rho = (hc*s_rho+Cs_r*h)/(hc+h)
        z_rho = zeta+(zeta+h)*Z_rho
        return z_rho
    

simulationpaths = sorted(glob.glob('data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/*avg*2006*'))+sorted(glob.glob('data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/*avg*2007*'))
simulation = []
for p in simulationpaths:
    print(p)
    x = load_croco(p, YORIG='1900-01-01 00:00:00',variables=['v','zeta','temp'], chunks=dict(time=1, eta_rho=-1, xi_rho=-1, s_rho=-1))
    y = load_croco(p, YORIG='1900-01-01 00:00:00',variables=['temp'], chunks=dict(time=1, eta_rho=-1, xi_rho=-1, s_rho=-1)).isel(s_rho=-1).rename({'temp':'sst'})
    simulation.append(xr.merge([x,y]))
# simulation      = [xr.merge([load_croco(p, YORIG='1900-01-01 00:00:00',variables=['v','zeta'], chunks=None).load(),
#                              load_croco(p, YORIG='1900-01-01 00:00:00', variables=['temp'], chunks=None).isel(s_rho=-1).load()])
#                    for p in simulationpaths]
simulation = xr.concat(simulation, 'time').where(grid.mask_rho==1).sortby('time')
simulation['lon_rho'] = (simulation['lon_rho']+180)%360-180
z_rho = rhopoints_depths(grid.h.expand_dims(dim={'s_rho':simulation.s_rho}), simulation.zeta, simulation.s_rho, Cs_r, hc)
z_rho = z_rho.transpose('time', 's_rho', 'eta_rho', 'xi_rho')

simulation.to_zarr('data/ZARR/testsim_BLKNOMASK', consolidated=True, mode='w')
z_rho.to_dataset(name='z_rho').chunk({'time':1, 'eta_rho':-1, 'xi_rho':-1, 's_rho':-1}).to_zarr('data/ZARR/testsim_BLKNOMASK', consolidated=True, mode='a')

# simulation.chunk({'time':1, 's_rho':50, 'eta_rho':292, 'xi_rho':324}).to_zarr('data/ZARR/testsim1', consolidated=True, mode='w')
# z_rho.to_dataset(name='z_rho').chunk({'time':1, 's_rho':50, 'eta_rho':292, 'xi_rho':324}).to_zarr('data/ZARR/testsim1', consolidated=True, mode='a')

data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M1.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M10.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M11.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M12.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M2.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M3.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M4.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M5.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M6.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M7.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M8.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2006M9.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2007M1.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2007M10.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2007M11.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMASK/testsim_avg_Y2007M12.nc
data/CROCO/OUTPUT/TESTSIM/BLK_NOMA

/ceaza/lucas/miniconda3/envs/main/lib/python3.8/site-packages/xarray/core/indexing.py:1234: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  value = value[(slice(None),) * axis + (subkey,)]
